- Better docstring
- Better naming
    - New method name should not be so similar with other methods. i.e `__getitem__` and `getitem`
- Internal & External design
- Make predictions

# TransformDict

In [86]:
# This is my implementation, not the one in Raymond's talk

class TransformDict(dict):
    
    def __init__(self, func):
        self.transform_func = func
        self.keymap = dict()
        
    def __setitem__(self, key, value):
        transformed = self.transform_func(key)
        self.keymap[transformed] = key
        super().__setitem__(transformed, value)
        
    def __getitem__(self, key):
        transformed = self.transform_func(key)
        return super().__getitem__(transformed)
    
    def getitem(self, key):
        transformed = self.transform_func(key)
        return (self.keymap[transformed], super().__getitem__(transformed))

In [87]:
d = TransformDict(str.lower)

In [88]:
d['Foo'] = 5

In [89]:
d['fOo']

5

In [90]:
set(d.keys())

{'foo'}

In [91]:
d.items()

dict_items([('foo', 5)])

In [92]:
d.getitem('foo')

('Foo', 5)

# Non-underscore Methods

In [93]:
[name for name in dir(d) if not name.startswith('_')]

['clear',
 'copy',
 'fromkeys',
 'get',
 'getitem',
 'items',
 'keymap',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'transform_func',
 'update',
 'values']

In [94]:
d.getitem('foo')

('Foo', 5)

# Design

## Internal Design

In short, it created a single, dict-like tool that combined:

- A transformation function such as `str.casefold`
- A first dictionary: `transformed key --> value`
- A second dictionary: `transformed key --> original_untransformed_key`

## External Design

Various API choices were made:

- The transformation function was stored in a read-only attribute
- The two internal dicts are not exposed
- The combined dict modeled: `untransformed_key -> value`
- The `items` method modeled: `[{original_untransformed_key, current_value), ...]`
- The new `getitem` method modeled: `untransformed key --> (original_untransformed_key, current_value)`
- The `__missing__` method was not supported


# Make Predictions

In [95]:
count = 0

def casefold(s):
    global count
    count += 1
    return str.casefold(s)

def reset():
    global count
    count = 0
    
def show():
    print(f'{count} calls')

In [96]:
d = TransformDict(casefold)

In [97]:
reset(); show();

0 calls


In [98]:
d['Raymond'] = 'red'

In [99]:
show()

1 calls


In [100]:
d['RacHel'] = 'blue'

In [101]:
show()

2 calls


In [102]:
d['Rachel'] = 'green'
show()

3 calls


In [103]:
d['Rachel']

'green'

In [104]:
show()

4 calls


In [105]:
d.items()

dict_items([('raymond', 'red'), ('rachel', 'green')])

In [106]:
show()

4 calls


In [108]:
reset();
e = TransformDict(int)

In [109]:
e['12'] = 'twelve'
e['13'] = 'thirteen'

In [111]:
e[12]

'twelve'

In [113]:
e[12.0]

'twelve'

In [114]:
e[b'12']

'twelve'

In [117]:
e[8]

KeyError: 8

Design fault: should be KeyError instead of ValueError

In [118]:
e['abcd']

ValueError: invalid literal for int() with base 10: 'abcd'

In [120]:
for num in [12, 12.0, b'12', 8, 'hello']:
    print(f'Looking up: {num}')
    try:
        print(f'---> {e[num]}')
    except KeyError:
        print(f'---> {num} is not in the dictionary')

Looking up: 12
---> twelve
Looking up: 12.0
---> twelve
Looking up: b'12'
---> twelve
Looking up: 8
---> 8 is not in the dictionary
Looking up: hello


ValueError: invalid literal for int() with base 10: 'hello'